# Test each component for the indexer

In [1]:
from urllib.parse import urlsplit, urljoin
from dotenv import load_dotenv
from collections import deque
from bs4 import BeautifulSoup
import requests
import socket
import boto3
import json
import re
import os
import hashlib
import urllib.robotparser
from urllib.parse import urlparse
import time

# S3 Access

In [7]:
from dotenv import load_dotenv

In [10]:
load_dotenv(dotenv_path='../.env')
bucket_name = os.getenv("S3_BUCKET_NAME")
aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")
concurrency = int(os.getenv("CR_REQUESTS_CONCURRENCY", default=1))
timeout = int(os.getenv("CR_REQUESTS_TIMEOUT", default=20))

In [14]:
s3 = boto3.resource('s3')
for bucket in s3.buckets.all():
    print(bucket.name)

bingo-crawling
bingo-meta
contectresearch
contectresearch-lambda
contectresearchnew
contectresearchtest
dog-processor-dev-serverlessdeploymentbucket-ih7gh7guil90
dog-processor-dev-serverlessdeploymentbucket-mikqvl2w0wuz
elasticbeanstalk-us-east-1-583566540209
find-feature-vector-deployment-lambda
image-uploader-dev-serverlessdeploymentbucket-ill8mcgtm25z
image-uploader-dev-serverlessdeploymentbucket-o62lum340oop
ndpainteddogretrain
ndpainteddogs
ndprocessedimages
ndwild-painteddogs-bryan-serverlessdeploymentbuck-d5wwbg5acpe6
ndwild-painteddogs-bryantest-de-painteddogsbucket-1vc1iyczixlgp
ndwild-painteddogs-dev-painteddogsbucket-1qvbs53odxdvy
ndwild-painteddogs-dev-serverlessdeploymentbucket-vd75c82pfwjw
ndwild-painteddogs-prod-painteddogsbucket-1a1gqmlfu9ge7
ndwild-painteddogs-prod-serverlessdeploymentbucke-13yv8eq0ybkpa
notes-app-api-prod-serverlessdeploymentbucket-4l95sv4nq8s2
notes-app-jack-d-api-pro-serverlessdeploymentbuck-4yc2j0id6f6y
notes-app-jack-d-uploads
notes-app-trial-clie

In [21]:
my_bucket = s3.Bucket(bucket_name)

file_key_list = []

for my_bucket_object in my_bucket.objects.all():
    file_key_list.append(my_bucket_object.key)

    

['0b2c2b9a5ea7243d5f00ad7b9da1b51dfd80cb66', '0f34b436db02e041c9092c7163bd01b86fb01700', '0f40bd8d5d6e186167991a0cdc0ef25eb4012ff8', '18e94bff1c471a8504d4f9205060a08f15e9627a', '1ef1443faf224d8bd18acd1ad398c7f4a1a272f2', '26d37a06875101d699bb09ed1e06bee1cd66e103', '5721b999ce9ba04ee8332a9032ba0ed2401b4307', '64a99738bee5af17a0a2bb2e1b71a26ac279b74c', '6531c3d614180631cbc44463cde8f4d5b7abd546', '677538ba119ca06c8e266ded62df1564cb2232a4', '6bc1c9e0091b303be0d696e308c9347492b3a197', '6bec4783da908d4b8277c51b1e3be48af7145b7c', '6c489364b91bccaf10914caf0eec563cfb167631', '7476ed58fe566f0f651479dd263ce067e2e3e97b', '87ed5625e3e7db21d4047dac0b81a7defe2dc7c5', '99c395fae3d1a5bbf378b11d3e40d23aa2358a9f', 'd68289fd96589f0be369ab7f34bc79057bd42cad', 'e596520a4a52ff0958ec1036cbcaaaf86f02b421']


In [29]:
nb_file = 1

for i in range(nb_file):
    file_key = file_key_list[i]
    obj = s3.Object(bucket_name, file_key)
    file = obj.get()['Body'].read().decode('utf-8')

In [30]:
file

'<!DOCTYPE html>\n<html class="client-nojs" dir="ltr" lang="en">\n <head>\n  <meta charset="utf-8"/>\n  <title>\n   Portal:Society - Wikipedia\n  </title>\n  <script>\n   document.documentElement.className="client-js";RLCONF={"wgCanonicalNamespace":"Portal","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":100,"wgPageName":"Portal:Society","wgTitle":"Society","wgCurRevisionId":920678291,"wgRevisionId":920678291,"wgArticleId":2076539,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Portals with short description","Wikipedia semi-protected portals","Wikipedia move-protected portals","Portals with undated maintenance templates","Portals with untriaged subpages","All portals with untriaged subpages","All portals","Portals with no named maintainer","Random portal component with 31–40 available subpages","Random portal component with 31–40 available image subpages","Random portal component with 16–20 available subpages","Past feat

# Text processing

In [40]:
#  [DONE] Lower case, remove numbers and diacritics;
#  [DONE] Remove punctuation and whitespaces
#  [DONE] Tokenization
#  [DONE] Remove stop words [2]
#  [DONE] Stemming
#  [] Lemmatization

In [2]:
import codecs

In [3]:
# Choose an example HTML file and get only the text from it
exmaple_file_path = "/Users/kiyoshi/Desktop/Christian Poellabauer (Notre Dame).html"
file = codecs.open(exmaple_file_path, 'r')
document= BeautifulSoup(file.read()).get_text()
print(document)



Christian Poellabauer (Notre Dame)















Christian Poellabauer
Professor
              Computer Science and Engineering
              University of Notre Dame
              323B Cushing Hall
              Notre Dame, IN 46556
              Phone: (574) 631-9131
              Fax: (574) 631-9260
              Email: cpoellab AT nd.edu










 Education

Ph.D., Georgia Institute of Technology, May 2004
Diplom-Ingenieur, University of Technology Vienna, June 1998

 Research

My research interests span areas such as distributed real-time systems, resource
management (e.g., energy management), wireless/mobile networks, 
wireless sensor networks, vehicular networks, smart phone apps, and pervasive
healthcare applications and systems.
I direct the Mobile Computing Lab (M-Lab), where we focus on projects such as mobile applications and services, wireless networks (primarily MANETs, VANETs, mesh networks, etc.), wireless sensor networks, QoS and real-time requirements, energy effi

In [5]:
# Delete the empty lines and tokenizing
import nltk
from nltk.tokenize import word_tokenize

In [10]:
st = ""

docwords = nltk.sent_tokenize(document)

for line in docwords:
    print(line)
    print("----")
    line = (line.rstrip())
    if line:
        if re.match("^[A-Za-z]*$",line):
            #if (line not in stop and len(line)>1):
            st = st + " " + line

# All lower cases
st_lower = st.lower()
print(st_lower)



Christian Poellabauer (Notre Dame)















Christian Poellabauer
Professor
              Computer Science and Engineering
              University of Notre Dame
              323B Cushing Hall
              Notre Dame, IN 46556
              Phone: (574) 631-9131
              Fax: (574) 631-9260
              Email: cpoellab AT nd.edu










 Education

Ph.D., Georgia Institute of Technology, May 2004
Diplom-Ingenieur, University of Technology Vienna, June 1998

 Research

My research interests span areas such as distributed real-time systems, resource
management (e.g., energy management), wireless/mobile networks, 
wireless sensor networks, vehicular networks, smart phone apps, and pervasive
healthcare applications and systems.
----
I direct the Mobile Computing Lab (M-Lab), where we focus on projects such as mobile applications and services, wireless networks (primarily MANETs, VANETs, mesh networks, etc.
----
), wireless sensor networks, QoS and real-time requirements, 

In [26]:
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from nltk.corpus import stopwords

In [35]:
word_list = st_lower.split(' ')

In [36]:
word_list

['',
 'christian',
 'poellabauer',
 'notre',
 'dame',
 'christian',
 'poellabauer',
 'professor',
 'computer',
 'science',
 'and',
 'engineering',
 'university',
 'of',
 'notre',
 'dame',
 'cushing',
 'hall',
 'notre',
 'dame',
 'in',
 'phone',
 'fax',
 'email',
 'cpoellab',
 'at',
 'education',
 'georgia',
 'institute',
 'of',
 'technology',
 'may',
 'university',
 'of',
 'technology',
 'vienna',
 'june',
 'research',
 'my',
 'research',
 'interests',
 'span',
 'areas',
 'such',
 'as',
 'distributed',
 'systems',
 'resource',
 'management',
 'energy',
 'management',
 'networks',
 'wireless',
 'sensor',
 'networks',
 'vehicular',
 'networks',
 'smart',
 'phone',
 'apps',
 'and',
 'pervasive',
 'healthcare',
 'applications',
 'and',
 'systems',
 'i',
 'direct',
 'the',
 'mobile',
 'computing',
 'lab',
 'where',
 'we',
 'focus',
 'on',
 'projects',
 'such',
 'as',
 'mobile',
 'applications',
 'and',
 'services',
 'wireless',
 'networks',
 'primarily',
 'manets',
 'vanets',
 'mesh',
 'net

In [37]:
filtered_list = [word for word in word_list if word not in stopwords.words('english')]

In [38]:
filtered_list

['',
 'christian',
 'poellabauer',
 'notre',
 'dame',
 'christian',
 'poellabauer',
 'professor',
 'computer',
 'science',
 'engineering',
 'university',
 'notre',
 'dame',
 'cushing',
 'hall',
 'notre',
 'dame',
 'phone',
 'fax',
 'email',
 'cpoellab',
 'education',
 'georgia',
 'institute',
 'technology',
 'may',
 'university',
 'technology',
 'vienna',
 'june',
 'research',
 'research',
 'interests',
 'span',
 'areas',
 'distributed',
 'systems',
 'resource',
 'management',
 'energy',
 'management',
 'networks',
 'wireless',
 'sensor',
 'networks',
 'vehicular',
 'networks',
 'smart',
 'phone',
 'apps',
 'pervasive',
 'healthcare',
 'applications',
 'systems',
 'direct',
 'mobile',
 'computing',
 'lab',
 'focus',
 'projects',
 'mobile',
 'applications',
 'services',
 'wireless',
 'networks',
 'primarily',
 'manets',
 'vanets',
 'mesh',
 'networks',
 'etc',
 'wireless',
 'sensor',
 'networks',
 'qos',
 'requirements',
 'energy',
 'efficiency',
 'healthcare',
 'solutions',
 'group',
 

In [39]:
print(len(word_list))
print(len(filtered_list))

977
742


In [44]:
# Stemming and Lemmatization
from nltk.stem.snowball import SnowballStemmer

stem_list = []
stemmer = SnowballStemmer(language='english')
for word in filtered_list:
    stem_list.append(stemmer.stem(word))

In [46]:
stem_list[:20]

['',
 'christian',
 'poellabau',
 'notr',
 'dame',
 'christian',
 'poellabau',
 'professor',
 'comput',
 'scienc',
 'engin',
 'univers',
 'notr',
 'dame',
 'cush',
 'hall',
 'notr',
 'dame',
 'phone',
 'fax']

In [62]:
from nltk.stem import WordNetLemmatizer

In [63]:
wordnet_lemmatizer = WordNetLemmatizer()

In [65]:
for word in stem_list:
    print (word, wordnet_lemmatizer.lemmatize(word, pos="v"))

 
christian christian
poellabau poellabau
notr notr
dame dame
christian christian
poellabau poellabau
professor professor
comput comput
scienc scienc
engin engin
univers univers
notr notr
dame dame
cush cush
hall hall
notr notr
dame dame
phone phone
fax fax
email email
cpoellab cpoellab
educ educ
georgia georgia
institut institut
technolog technolog
may may
univers univers
technolog technolog
vienna vienna
june june
research research
research research
interest interest
span span
area area
distribut distribut
system system
resourc resourc
manag manag
energi energi
manag manag
network network
wireless wireless
sensor sensor
network network
vehicular vehicular
network network
smart smart
phone phone
app app
pervas pervas
healthcar healthcar
applic applic
system system
direct direct
mobil mobil
comput comput
lab lab
focus focus
project project
mobil mobil
applic applic
servic servic
wireless wireless
network network
primarili primarili
manet manet
vanet vanet
mesh mesh
network network
etc 

In [32]:
from nltk.stem import WordNetLemmatizer 
  
lemmatizer = WordNetLemmatizer()

print("rocks :", lemmatizer.lemmatize("rocks")) 
print("corpora :", lemmatizer.lemmatize("corpora")) 

rocks : rock
corpora : corpus


In [53]:
# Convert the list back to string
stem_str = ' '.join(stem_list)

In [55]:
stem_str[:100]

' christian poellabau notr dame christian poellabau professor comput scienc engin univers notr dame c'

In [58]:
stem_str_lemma = sp(stem_str)

In [59]:
lemm_list = []

for word in stem_str_lemma:
    lemm_list.append(word.lemma_)

In [61]:
for word in stem_str_lemma:
    print(word.text + '  ===>', word.lemma_)

   ===>  
christian  ===> christian
poellabau  ===> poellabau
notr  ===> notr
dame  ===> dame
christian  ===> christian
poellabau  ===> poellabau
professor  ===> professor
comput  ===> comput
scienc  ===> scienc
engin  ===> engin
univers  ===> univer
notr  ===> notr
dame  ===> dame
cush  ===> cush
hall  ===> hall
notr  ===> notr
dame  ===> dame
phone  ===> phone
fax  ===> fax
email  ===> email
cpoellab  ===> cpoellab
educ  ===> educ
georgia  ===> georgia
institut  ===> institut
technolog  ===> technolog
may  ===> may
univers  ===> univer
technolog  ===> technolog
vienna  ===> vienna
june  ===> june
research  ===> research
research  ===> research
interest  ===> interest
span  ===> span
area  ===> area
distribut  ===> distribut
system  ===> system
resourc  ===> resourc
manag  ===> manag
energi  ===> energi
manag  ===> manag
network  ===> network
wireless  ===> wireless
sensor  ===> sensor
network  ===> network
vehicular  ===> vehicular
network  ===> network
smart  ===> smart
phone  ===> 

In [60]:
lemm_list

[' ',
 'christian',
 'poellabau',
 'notr',
 'dame',
 'christian',
 'poellabau',
 'professor',
 'comput',
 'scienc',
 'engin',
 'univer',
 'notr',
 'dame',
 'cush',
 'hall',
 'notr',
 'dame',
 'phone',
 'fax',
 'email',
 'cpoellab',
 'educ',
 'georgia',
 'institut',
 'technolog',
 'may',
 'univer',
 'technolog',
 'vienna',
 'june',
 'research',
 'research',
 'interest',
 'span',
 'area',
 'distribut',
 'system',
 'resourc',
 'manag',
 'energi',
 'manag',
 'network',
 'wireless',
 'sensor',
 'network',
 'vehicular',
 'network',
 'smart',
 'phone',
 'app',
 'pervas',
 'healthcar',
 'applic',
 'system',
 'direct',
 'mobil',
 'comput',
 'lab',
 'focus',
 'project',
 'mobil',
 'applic',
 'servic',
 'wireless',
 'network',
 'primarili',
 'manet',
 'vanet',
 'mesh',
 'network',
 'etc',
 'wireless',
 'sensor',
 'network',
 'qos',
 'requir',
 'energi',
 'effici',
 'healthcar',
 'solut',
 'group',
 'research',
 'fund',
 'grant',
 'nsf',
 'includ',
 'career',
 'award',
 'offic',
 'naval',
 'resear

# 1. Connect to AWS S3

In [3]:
s3 = boto3.resource('s3')

In [4]:
S3_BUCKET_NAME = "bingo-crawling"

In [5]:
bucket = s3.Bucket(S3_BUCKET_NAME)